In [20]:
!python3 --version

Python 3.13.2


In [21]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
import pickle

In [22]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc_greentaxi_tripdata_experiment')

<Experiment: artifact_location='/Users/mac/Projects/MLops_datatalks/mlruns/1', creation_time=1748148094528, experiment_id='1', last_update_time=1748148094528, lifecycle_stage='active', name='nyc_greentaxi_tripdata_experiment', tags={}>

In [23]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime)
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)
    df = df[((df['duration'] > 1) & (df['duration'] <= 60))]
    
    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [24]:
df_train = read_dataframe('green_tripdata/green_tripdata_2021-01.parquet')
df_val = read_dataframe('green_tripdata/green_tripdata_2021-02.parquet')

In [25]:
df_train.shape, df_val.shape

((73833, 21), (61889, 21))

In [26]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_'  +  df_val['DOLocationID']

In [27]:
df_train.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration,PU_DO
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,...,0.00,0.0,None,0.3,6.80,2.0,1.0,0.00,3.933333,43_151
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,...,2.81,0.0,None,0.3,16.86,1.0,1.0,2.75,8.750000,166_239
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,...,1.00,0.0,None,0.3,8.30,1.0,1.0,0.00,5.966667,41_42
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,...,0.00,0.0,None,0.3,9.30,2.0,1.0,0.00,7.083333,168_75
7,2,2021-01-01 00:26:31,2021-01-01 00:28:50,N,1.0,75,75,6.0,0.45,3.5,...,0.96,0.0,None,0.3,5.76,1.0,1.0,0.00,2.316667,75_75


In [28]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

y_train = df_train['duration'].values
y_val = df_val['duration'].values

In [29]:
X_train, y_train
X_val, y_val

(<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 118522 stored elements and shape (61889, 13221)>,
 array([17.91666667,  6.5       , 15.25      , ..., 14.        ,
         7.        ,  7.        ], shape=(61889,)))

In [33]:
from math import sqrt

In [36]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

rmse_lr = sqrt(mean_squared_error(y_val, y_pred))
rmse_lr

7.756932266972238

In [38]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [42]:
with mlflow.start_run():
    mlflow.set_tag("engineer", "richkinwe")

    mlflow.log_param("train_data_path", "green_tripdata/green_tripdata_2021-01.parquet")
    mlflow.log_param("val_data_path", "green_tripdata/green_tripdata_2021-02.parquet")

    alpha = 0.00001
    mlflow.log_param("alpha", alpha)
    ls = Lasso(alpha)
    ls.fit(X_train, y_train)

    y_pred = ls.predict(X_val)

    rmse_ls = sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse_ls)

In [ ]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
val = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("engineer", "richkinwe")
        mlflow.log_param("train_data_path", "green_tripdata/green_tripdata_2021-01.parquet")
        mlflow.log_param("val_data_path", "green_tripdata/green_tripdata_2021-02.parquet")
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(val, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(val)
        rmse = sqrt(mean_squared_error(y_val, y_pred))
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)